In [3]:
import zipfile
from pathlib import Path
import partitura
import pandas as pd
import xml.etree.ElementTree as ET
from spacy import displacy
import numpy as np

In [7]:
# delete all mus files in all subfolders
for file in Path('data').rglob("*"):
    if file.suffix == '.mus' or file.suffix == '.mid' or file.suffix == '.kern':
        file.unlink()

In [5]:
#unzip all files in zips folder

# for zip_file in Path("zips").iterdir():
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(Path("unzipped"))

In [7]:
# check if ts trees are in unzipped folder
# for folder in Path("unzipped").iterdir():
#     if not any([file.name.startswith("TS") for file in folder.iterdir()]):
#         print("not a TS tree in ", folder)

In [ ]:
# check if there are scores everywhere
# for folder in Path("unzipped").iterdir():
#     msc_file = [file for file in folder.iterdir() if file.name.startswith("MSC")]
#     numbered_file = [file for file in folder.iterdir() if file.name.startswith(folder.name[:2])]
#     assert(len(msc_file)+len(numbered_file) == 1)
#     score_file = msc_file[0] if len(msc_file) == 1 else numbered_file[0]
#     try:
#         score = partitura.load_musicxml(str(score_file))
#     except Exception as e:
#         print("error loading ", score_file, e)

In [13]:
# build a pandas dataframe with all the scores paths
def get_score_path(folder):
    score_files = [file for file in folder.iterdir() if file.name.startswith("score")]
    assert(len(score_files) == 1)
    return str(score_files[0])

def get_ts_path(folder):
    ts_file = [file for file in folder.iterdir() if file.name.startswith("TS")]
    assert(len(ts_file) == 1)
    return str(ts_file[0])

def get_title(folder):
    title_file = [file for file in folder.iterdir() if file.suffix == ".txt"]
    assert(len(title_file) == 1)
    return title_file[0].name

list_of_tuples = []

for folder in Path("data").iterdir():
    list_of_tuples.append((get_title(folder),get_score_path(folder), get_ts_path(folder)))

df = pd.DataFrame(list_of_tuples, columns = ['title', 'score', 'ts'])

In [14]:
df

,title,score,ts
0,01_Waltz in E flat Grande Valse Brillante Op.1...,data\01\score.xml,data\01\TS.xml
1,02_Moments Musicaux.txt,data\02\score.xml,data\02\TS.xml
2,03_Bagatelle 'Fur Elise' WoO.59.txt,data\03\score.xml,data\03\TS.xml
3,04_The Preludes Op.28 No.15.txt,data\04\score.xml,data\04\TS.xml
4,05_Turkish March.txt,data\05\score.xml,data\05\TS.xml
...,...,...,...
295,95_12 Variationen uber ein franzosisches Lied ...,data\95\score.xml,data\95\TS.xml
296,96_Lieder ohne Worte Heft 5 Op.62-6 Fruhlingsl...,data\96\score.xml,data\96\TS.xml
297,97_Les Patineurs Op.183.txt,data\97\score.xml,data\97\TS.xml
298,98_Sonate fur Klavier Nr.8 c moll Pathetique O...,data\98\score.xml,data\98\TS.xml


# Parse tree annotations from the xml file

In [3]:
# parse the xml file with annotations in a list of dependencies
tree = ET.parse(r'data\07\TS-07.xml')
root = tree.getroot()

In [45]:
# def iterative_parse(xml_elem):
#     primary_children = xml_elem.find("ts").find("primary")
#     secondary_children = xml_elem.find("ts").find("secondary")
#     # recursion ending condition
#     if primary_children is None:
#         assert secondary_children is None
#         return xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
#     else:
#         assert secondary_children is not None
#         return [iterative_parse(primary_children), iterative_parse(secondary_children)]


# iterative_parse(root)

In [4]:
def iterative_parse(xml_elem):
    primary_children = xml_elem.find("ts").find("primary")
    secondary_children = xml_elem.find("ts").find("secondary")
    if primary_children is None: # recursion ending condition
        assert secondary_children is None
        return [], xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
    else: # recursive call
        assert secondary_children is not None
        out_list = [] # dependency list
        iterative_result_primary = iterative_parse(primary_children)
        iterative_result_secondary = iterative_parse(secondary_children)
        # merge the dependencies lists computed deeper
        out_list.extend(iterative_result_primary[0])
        out_list.extend(iterative_result_secondary[0])
        # append the dependency for the current node
        out_list.append((iterative_result_primary[1], iterative_result_secondary[1]))
        # return the dependency list, and the id of the current node, i.e., the primary
        return out_list, iterative_parse(primary_children)[1]


arcs = iterative_parse(root)[0]
arcs

[('P1-9-5', 'P1-9-4'),
 ('P1-9-7', 'P1-9-6'),
 ('P1-9-5', 'P1-9-7'),
 ('P1-9-9', 'P1-9-8'),
 ('P1-9-5', 'P1-9-9'),
 ('P1-9-10', 'P1-9-5'),
 ('P1-9-1', 'P1-9-2'),
 ('P1-9-1', 'P1-9-3'),
 ('P1-9-10', 'P1-9-1'),
 ('P1-8-5', 'P1-8-4'),
 ('P1-8-6', 'P1-8-5'),
 ('P1-8-2', 'P1-8-3'),
 ('P1-8-1', 'P1-7-15'),
 ('P1-8-2', 'P1-8-1'),
 ('P1-8-6', 'P1-8-2'),
 ('P1-9-10', 'P1-8-6'),
 ('P1-6-1', 'P1-5-12'),
 ('P1-6-9', 'P1-6-8'),
 ('P1-6-7', 'P1-6-6'),
 ('P1-6-2', 'P1-6-3'),
 ('P1-6-4', 'P1-6-5'),
 ('P1-6-2', 'P1-6-4'),
 ('P1-6-7', 'P1-6-2'),
 ('P1-6-9', 'P1-6-7'),
 ('P1-6-1', 'P1-6-9'),
 ('P1-6-15', 'P1-6-14'),
 ('P1-6-12', 'P1-6-13'),
 ('P1-6-11', 'P1-6-12'),
 ('P1-6-15', 'P1-6-11'),
 ('P1-7-1', 'P1-6-15'),
 ('P1-6-1', 'P1-7-1'),
 ('P1-7-13', 'P1-7-12'),
 ('P1-7-14', 'P1-7-13'),
 ('P1-7-3', 'P1-7-2'),
 ('P1-7-5', 'P1-7-4'),
 ('P1-7-3', 'P1-7-5'),
 ('P1-7-11', 'P1-7-10'),
 ('P1-7-7', 'P1-7-6'),
 ('P1-7-9', 'P1-7-8'),
 ('P1-7-7', 'P1-7-9'),
 ('P1-7-11', 'P1-7-7'),
 ('P1-7-3', 'P1-7-11'),
 ('P1-7-14',

In [6]:
import numpy as np
note_ids = list(np.unique([e.attrib['id'] for e in root.findall('''.//note''')]))

In [7]:
spacy_words = [{"text": word, "tag": ""} for word in note_ids]
spacy_arcs = []
for (start_id, end_id) in arcs:
    start_ix = note_ids.index(start_id)
    end_ix = note_ids.index(end_id)
    if start_ix < end_ix:
        spacy_arcs.append({"start": start_ix, "end": end_ix, "label": "", "dir": "right"})
    else:
        spacy_arcs.append({"start": end_ix, "end": start_ix, "label": "", "dir": "left"})


spacy_dict = {"words": spacy_words, "arcs": spacy_arcs}
displacy.render(spacy_dict, style='dep', jupyter=True, manual=True)

In [18]:
import spacy
from spacy import displacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

sentence = 'Deemed universities charge huge fees'

# nlp function returns an object with individual token information, 
# linguistic features and relationships
doc = nlp(sentence)

print ("{:<15} | {:<8} | {:<15} | {:<20}".format('Token','Relation','Head', 'Children'))
print ("-" * 70)

for token in doc:
  # Print the token, dependency nature, head and all dependents of the token
  print ("{:<15} | {:<8} | {:<15} | {:<20}"
         .format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
  
# Use displayCy to visualize the dependency 
displacy.render(doc, style='dep', jupyter=True, options={'distance': 120})

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Deemed          | amod     | universities    | []                  
universities    | nsubj    | charge          | [Deemed]            
charge          | ROOT     | charge          | [universities, fees]
huge            | amod     | fees            | []                  
fees            | dobj     | charge          | [huge]              


In [22]:
l = []
for token in doc:
    l.append(token)
type(doc)

spacy.tokens.doc.Doc

In [31]:
obj = {
    "words": [
        {"text": "This", "tag": ""},
        {"text": "is", "tag": ""},
        {"text": "a", "tag": ""},
        {"text": "sentence", "tag": ""}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "", "dir": "right"},
        {"start": 2, "end": 3, "label": "", "dir": "left"},
        {"start": 1, "end": 3, "label": "", "dir": "right"}
    ]
}

displacy.render(obj, style='dep', jupyter=True, options={'distance': 120}, manual=True)

# Parse note features from score file

In [11]:
score_path = df.iloc[0]["score"]

In [21]:
score = partitura.load_musicxml(score_path)
len(score.parts[0].rests)

0

In [25]:
extended_score_note_array = partitura.utils.music.ensure_notearray(
    score,
    include_metrical_position=True, # adds 3 fields: is_downbeat, rel_onset_div, tot_measure_div
    include_time_signature=True, # adds 2 fields: time_signature_numerator, time_signature_denominator
)

In [33]:
np.char.array(extended_score_note_array["ts_beats"]) + np.char.array(extended_score_note_array["ts_beat_type"])

chararray([b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34'], dtype='|S2')

In [35]:
scores = [partitura.load_musicxml(score_path) for score_path in df["score"]]  
    

c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "sentito" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "Largamente" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cresc.  -   -   -  " (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cre    -     -     -     scen     -    -    -     do" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_re

In [64]:
all_time_signatures = []

for i,score in enumerate(scores):
    na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,        
    )
    # if len(score.parts[0].rests) > 0:
    #     print(score_path, len(score.parts[0].rests))
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    all_time_signatures.append(np.unique(time_signatures)[-1])
    if np.unique(time_signatures).shape[0] > 1:
        print(np.unique(time_signatures), df["score"][i], np.unique(time_signatures)[-1] )

['1/8' '2/8' '3/8'] data\03\03_Bagatelle 'Fur Elise' WoO.59.xml 3/8
['1/8' '5/8' '6/8'] data\06\06_Blumenlied Op.39.xml 6/8
['1/8' '11/8' '12/8'] data\07\07_Nocturne.xml 12/8
['1/4' '3/4' '4/4'] data\09\09_String Quartet in F major Op.3 No.5 Serenade.xml 4/4
['1/4' '2/4' '3/4'] data\10\10_Wiegenlied.xml 3/4
['1/4' '4/4'] data\11\11_Solvejgs Lied.xml 4/4
['2/4' '3/4'] data\115\MSC-115.xml 3/4
['1/4' '2/4' '3/4'] data\12\12_Anitras Dans.xml 3/4
['1/4' '3/4' '4/4'] data\13\13_Traumerei.xml 4/4
['1/4' '2/4' '3/4'] data\14\14_Menuett No.2 in G maj.xml 3/4
['1/4' '3/4' '4/4'] data\16\16_William Tell Overture.xml 4/4
['1/4' '2/4' '3/4'] data\19\19_Tannhauser Overture.xml 3/4
['1/8' '2/8' '3/8'] data\20\20_La Traviata Brindisi.xml 3/8
['1/8' '5/8' '6/8'] data\21\21_Plaisir d'Amour.xml 6/8
['2/4' '4/4'] data\25\25_L'arlesienne Suite No.2 Farandole.xml 4/4
['1/4' '2/2' '3/4'] data\37\37_Sonate fur Klavier Nr.48 C dur Op.30-1 Mov.1.xml 3/4
['1/4' '2/4' '3/4'] data\49\49_The Planets Op.32 Jupiter,

In [66]:
np.unique(all_time_signatures)
# for i, ts in enumerate(all_time_signatures):
#     print(ts, df["score"][i])

array(['12/8', '2/2', '2/4', '3/2', '3/4', '3/8', '4/4', '6/4', '6/8',
       '9/8'], dtype='<U4')

In [62]:
np.char.array([","]*na.shape[0]) + np.char.array(na["ts_beat_type"].astype(str))

chararray([',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4'], dtype='<U12')

In [74]:
score_path = df.iloc[6]["score"]
score = partitura.load_musicxml(score_path)

In [110]:
na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,    
    )

na, real_ts,real_measure_duration = correct_metrical_information(na)

na
# na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None', 12, 8, 12, 1, 22, 24, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8, 12, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8, 12, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8, 12, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8, 12, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8, 12, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (22. , 2.

In [109]:
def correct_metrical_information(na):
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    # get real time signature and measure duration, discarding pickup and ending measure ts
    real_ts = np.unique(time_signatures)[-1]
    real_measure_duration = na[time_signatures == real_ts][0]["tot_measure_div"]
    # find pickup notes
    pickup_note_indices = np.where((na["tot_measure_div"]!= real_measure_duration) * (na["onset_div"] < real_measure_duration ))[0]
    # set the pickup note to the correct metrical position
    na["rel_onset_div"][pickup_note_indices] = na["onset_div"][pickup_note_indices] + real_measure_duration - na["tot_measure_div"][pickup_note_indices]
    # set the measure duration to correct value
    na["tot_measure_div"] = real_measure_duration
    # set the correct time signature
    na["ts_beats"] = real_ts.split("/")[0]
    na["ts_mus_beats"] = real_ts.split("/")[0]
    na["ts_beat_type"] = real_ts.split("/")[1]
    return na, real_ts, real_measure_duration

def get_note_features(na):
    pitch = na["pitch"]
    metrical =  na["is_downbeat"] #TODO: add more metrical info
    duration = na["duration_div"]/na["tot_measure_div"]
    # TODO: consider rests as lag from previous note
    return np.concatenate((pitch, metrical, duration), axis=1)


In [105]:
na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None',  1, 8,  1, 1, 22,  2, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8,  4, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8,  4, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8,  4, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8,  4, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8,  4, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (22. , 2.

# Test data loader

In [1]:
from musicparser.data_loading import TSDataset
import partitura as pt
from pathlib import Path

c:\Users\fosca\.conda\envs\mir_research\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = TSDataset(Path("data"))

!!!!! Error with 01_Waltz in E flat Grande Valse Brillante Op.18.txt expected np.ndarray (got list)
!!!!! Error with 02_Moments Musicaux.txt expected np.ndarray (got list)
!!!!! Error with 03_Bagatelle 'Fur Elise' WoO.59.txt expected np.ndarray (got list)
!!!!! Error with 04_The Preludes Op.28 No.15.txt expected np.ndarray (got list)
!!!!! Error with 05_Turkish March.txt expected np.ndarray (got list)
!!!!! Error with 06_Blumenlied Op.39.txt expected np.ndarray (got list)
!!!!! Error with 07_Nocturne.txt expected np.ndarray (got list)
!!!!! Error with 08_Spinnerlied Op.14 No.4.txt expected np.ndarray (got list)
!!!!! Error with 09_String Quartet in F major Op.3 No.5 Serenade.txt expected np.ndarray (got list)
!!!!! Error with 10_Wiegenlied.txt expected np.ndarray (got list)
!!!!! Error with 100_Sonate fur Klavier Nr.11 A dur K.331 K6.300i Mov.1.txt expected np.ndarray (got list)
!!!!! Error with 101_unknown.txt expected np.ndarray (got list)
!!!!! Error with 102_unknown.txt expected np

In [3]:
# a = np.zeros((2,5))
for e in dataset:
    print(e)
    break

(array([[70.        ,  1.        ,  0.33333333],
       [74.        ,  0.        ,  0.16666667],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [82.        ,  1.        ,  0.66666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  1.        ,  0.33333333],
       [84.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [80.        ,  1.        ,  0.33333333],
       [82.        ,  0.        ,  0.16

In [3]:
# score = pt.load_musicxml(r"data\127\MSC-127.xml") 
# score = pt.load_musicxml(r"data\07\07_Nocturne.xml")
score = pt.load_musicxml(r"data\176_1\MSC-176.xml")
onsets = score.parts[0].note_array()["onset_div"]

In [9]:
score.parts[0].measure_number_map(onsets)

array([1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9])

In [31]:
import torch
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
a_indices = torch.arange(len(a))
cart_prod = torch.cartesian_prod(a_indices,a_indices)
# remove self loops
cart_prod = cart_prod[cart_prod[:,0]!=cart_prod[:,1]]
# concatenate
out = torch.cat([a[cart_prod[:,0]],a[cart_prod[:,1]]],dim=-1)
out.view(-1)

tensor([1, 2, 3, 4, 5, 6, 1, 2, 3, 7, 8, 9, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9,
        7, 8, 9, 1, 2, 3, 7, 8, 9, 4, 5, 6])

tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 4, 5, 6],
        [1, 2, 3, 7, 8, 9],
        [4, 5, 6, 1, 2, 3],
        [4, 5, 6, 4, 5, 6],
        [4, 5, 6, 7, 8, 9],
        [7, 8, 9, 1, 2, 3],
        [7, 8, 9, 4, 5, 6],
        [7, 8, 9, 7, 8, 9]])